In [218]:
import datetime as dt
from typing import List, Optional
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from abc import ABC, abstractmethod

In [219]:
class Analisys:
    
    def __init__(self, source: str, chapter: str|None, analisys_name: str, analisys_cost:str, analisys_comment: str,):              

        self.source    = source
        self.chapter     = chapter
        self.analisys_name   = analisys_name
        self.analisys_cost   = analisys_cost
        self.analisys_comment  = analisys_comment

    def __repr__(self) -> str:
        return f"source = {self.source}, chapter = {self.chapter}, analisys_name = {self.analisys_name}, analisys_cost = {self.analisys_cost}, analisys_comment = {self.analisys_comment}"
    
    def to_dict(self) -> dict:
        return self.__dict__


#     def to_DB(self, ...):
#         ...

#     @classmethod
#     def from_DB(cls, ...):
#         ...


In [220]:
class WebDriver(ABC):

    def __init__(self):
        self.urls: List[str] = []
            
    def _scan_page(self, url:str):
        # Прописываем нестандартный заголовок, чтобы сайт не принял нас за бота, который парсит его данные :)
        headers = requests.utils.default_headers()
        headers.update({
                    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
            })

        response = requests.get(url=url, headers=headers)

        if not response.ok:
            raise ValueError("no content to parse")

        return BeautifulSoup(response.content.decode(), "lxml")            

    @abstractmethod
    def get_analisys(self) :  
        ...


In [221]:
#  самая актуальная версия
class DriverMedart(WebDriver):
    
    URL = "https://medart.by/price/"
    NAME = "medart.by/"

    def __init__(self):
       ...
    
    def get_analisys(self) :
        
        analisys_dict={}
        i=0
        soup = self._scan_page(self.URL)

        ID = ["sect_6", "sect_5", "sect_7", "sect_8", "sect_9", "sect_10", "sect_11", "sect_12", "sect_13", "sect_14", "sect_15", "sect_16", "sect_17", "sect_19", "sect_146", "sect_151"]
        
        for k in ID:
            tag = soup.find('div', class_='section_filter', id=f"{k}")
            # print(k)
            # print(type(tag),len(tag),tag[0])

            R=tag.find(class_= 'pricemain-title_left sticky').text.strip()
            V= tag.find(class_= 'title').text.strip()
            S = tag.find(class_='categoriesmain_price-table').find('table')
#             print(R,V)

            # Находим все строки таблицы, кроме первой (заголовочной)
            rows = S.find_all("tr")#[1:]
            for row in rows:
#                 Проверяем, есть ли у элемента row атрибут colspan
                if row.find("td").has_attr('colspan'):
                    V = row.find("td", class_="title").text
                    continue
    
#                 Иначе продолжаем как обычно
                
                name_elem = row.find("td", class_="name")
                if name_elem is None:
                    print("Элемент name не найден!")
                    continue
                name = name_elem.get_text(strip=True)
                print(R,V,name)

                link = row.find("td", class_="name").find('a').get('href')
                                
                price_elem = row.find("td", class_="price")
                if price_elem is None:
                    print("Элемент price не найден!")
                    continue
                price = price_elem.get_text(strip=True)
                                
                chapter = (R,V)
                analisys_name = name
                analisys_cost = price
                analisys_comment = "https://medart.by"+link
                analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                analisys_dict[i] = (analisys.to_dict())
                i+=1 
                print('I=',i)
                print(analisys_dict)

        return analisys_dict

m = DriverMedart()
m.get_analisys()
df_med_art = pd.DataFrame.from_dict(m.get_analisys().values())
pd.set_option('display.max_colwidth', None)
df_med_art

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,source,chapter,analisys_name,analisys_cost,analisys_comment
0,medart.by/,"(Лаборатория, Анализ крови)",Общий анализ крови из вены,24.17руб.,https://medart.by/services/analiz-krovi/obshchiy-analiz-krovi-iz-veny/
1,medart.by/,"(Лаборатория, Анализ крови)",Общий анализ крови из пальца,26.08руб.,https://medart.by/services/analiz-krovi/obshchiy-analiz-krovi-iz-paltsa/
2,medart.by/,"(Лаборатория, Анализ крови)",Анализ крови на группу и резус-фактор,13.92руб.,https://medart.by/services/analiz-krovi/gruppa-i-rezus-faktor/
3,medart.by/,"(Лаборатория, Анализ крови)",Анализ крови на глюкозу экспресс-методом,2.37руб.,https://medart.by/services/analiz-krovi/glyukoza-ekspress-metodom/
4,medart.by/,"(Лаборатория, Анализ мочи)",Анализ мочи по Нечипоренко,2.96руб.,https://medart.by/services/analiz-mochi/analiz-mochi-po-nechiporenko/
...,...,...,...,...,...
260,medart.by/,"(Педиатрия, )",Лечение кашля,Уточняйте по телефонуруб.,https://medart.by/services/pediatriya/lechenie-kashlya/
261,medart.by/,"(Педиатрия, )",Лечение простуды и ОРВИ,Уточняйте по телефонуруб.,https://medart.by/services/pediatriya/lechenie-prostudy-i-orvi/
262,medart.by/,"(Онкология, )",Консультация врача-онколога первой или высшей категории,"47,08руб.",https://medart.by/services/onkologiya/konsultatsiya-vracha-onkologa-pervoy-ili-vysshey-kategorii/
263,medart.by/,"(Онкология, )",Повторная консультация врача-онколога первой или высшей категории,"38,25руб.",https://medart.by/services/onkologiya/povtornaya-konsultatsiya-vracha-onkologa-pervoy-ili-vysshey-kategorii/


In [224]:
df_med_art.iloc[200:250]

,source,chapter,analisys_name,analisys_cost,analisys_comment
200,medart.by/,"(Терапия, )",Повторная консультация врача-терапевта первой или высшей категории,"38,25руб.",https://medart.by/services/terapiya/povtornaya-konsultatsiya-vracha-terapevta-pervoy-ili-vysshey-kategorii/
201,medart.by/,"(Терапия, )","Консультация врача-терапевта, кандидата медицинских наук","54,64руб.",https://medart.by/services/terapiya/konsultatsiya-vracha-terapevta-kandidata-meditsinskikh-nauk/
202,medart.by/,"(Терапия, )","Повторная консультация врача-терапевта, кандидата медицинских наук","45,81руб.",https://medart.by/services/terapiya/povtornaya-konsultatsiya-vracha-terapevta-kandidata-meditsinskikh-nauk/
203,medart.by/,"(Терапия, )",Диагностика и лечение ОРВИ,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-orvi/
204,medart.by/,"(Терапия, )",Диагностика и лечение синдрома хронической усталости,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-sindroma-khronicheskoy-ustalosti/
205,medart.by/,"(Терапия, )",Диагностика и лечение гипертонической болезни,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-gipertonicheskoy-bolezni/
206,medart.by/,"(Терапия, )",Диагностика и лечение заболеваний сердца и сосудов,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-zabolevaniy-serdtsa-i-sosudov/
207,medart.by/,"(Терапия, )",Диагностика и лечение заболеваний желудочно-кишечного тракта,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-zabolevaniy-zheludochno-kishechnogo-trakta/
208,medart.by/,"(Терапия, )",Диагностика и лечение анемий,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-anemiy/
209,medart.by/,"(Терапия, )",Диагностика и лечение заболеваний печени,Уточняйте по телефонуруб.,https://medart.by/services/terapiya/diagnostika-i-lechenie-zabolevaniy-pecheni/
